In [343]:
import numpy as np
import pandas as pd
import yfinance as yf

In [344]:
TAX_RATE = 0.2
RISK_FREE_RETURN = 0.025

def get_return_rate(df, shift):
    return np.power(1+((df-df.shift(shift))/df.shift(shift)), 1/shift)-1

pce = pd.read_csv('../data/PCEPI.csv', index_col='DATE')
pce = pce.iloc[::12, :].PCEPI.pct_change()
pce.index = pce.index.astype('datetime64[ns]')
pce.index.name = 'Date'

def adjust_for_inflation(df, close_col, rate_col):
    df_inf = df.join(pce)
    data = [df_inf.iloc[0][close_col]]
    for i in range(1, df_inf.shape[0]):
        data.append(data[i-1]*(1+df_inf.iloc[i][rate_col])*(1-df_inf.iloc[i].PCEPI))
    return pd.Series(data, index=df_inf.index)

def tax_returns(close, frequencies):
    data = {}
    for freq in frequencies:
        gain = close-close.shift(freq)
        r_tax = np.where(gain > 0, (gain*(1-TAX_RATE)), gain)/close.shift(freq)
        data[f'{freq}Y'] = np.power(1+r_tax, 1/freq)-1
    return pd.DataFrame(data, index=close.index)

def get_returns(df, frequencies=[1, 5, 10, 20]):
    return pd.DataFrame({f'{freq}Y': get_return_rate(df, freq) for freq in frequencies}, index=df.index)

def sharpe_ratio(returns):
    return (returns.mean()-RISK_FREE_RETURN)/returns.std()

def get_performance(pf, alloc, close_col, rate_col, verbose=False):
    pf_returns = get_returns(pf[0][close_col])*alloc[0]
    for i in range(1, len(pf)):
        pf_returns += get_returns(pf[i][close_col])*alloc[i]
    if verbose:
        print('Returns with no inflation')
        print(pf_returns.describe())

    inflation_adjusted = [adjust_for_inflation(pf[0], close_col, rate_col)]
    pf_returns = get_returns(inflation_adjusted[0])*alloc[0]
    for i in range(1, len(pf)):
        inflation_adjusted.append(adjust_for_inflation(pf[i], close_col, rate_col))
        pf_returns += get_returns(inflation_adjusted[i])*alloc[i]
    if verbose:
        print('Returns adjusted for inflation')
        print(pf_returns.describe())

    taxed_returns = [tax_returns(inflation_adjusted[0], [10, 20])]
    pf_returns = taxed_returns[0]*alloc[0]
    for i in range(1, len(pf)):
        taxed_returns.append(tax_returns(inflation_adjusted[i], [10, 20]))
        pf_returns += taxed_returns[i]*alloc[i]
    if verbose:
        print('Returns adjusted for inflation and taxes')
        print(pf_returns.describe())

    print(f'Sharpe ratio\n{sharpe_ratio(pf_returns)}')


In [345]:
df_bond = pd.read_csv('../data/bonds.csv', index_col='Year')
df_bond.index = pd.to_datetime(df_bond.index.astype(str)+'-01-01')
df_bond['Return'] = df_bond['Return'].str.rstrip('%').astype(float)/100
close = [100]
for i in range(1, df_bond.shape[0]):
    close.append(close[i-1]*(1+df_bond.iloc[i-1].Return))
df_bond['Close'] = close
df_bond.index.name = 'Date'
get_performance([df_bond], [1], 'Close', 'Return', verbose=True)

Returns with no inflation
              1Y         5Y        10Y        20Y
count  43.000000  39.000000  34.000000  24.000000
mean    0.074737   0.076253   0.079425   0.078148
std     0.068168   0.037440   0.028742   0.019555
min    -0.029200   0.021019   0.034769   0.045494
25%     0.028800   0.048788   0.057916   0.063070
50%     0.063000   0.072715   0.076047   0.074963
75%     0.097000   0.098126   0.103232   0.096482
max     0.326000   0.184277   0.140963   0.106107
Returns adjusted for inflation
              1Y         5Y        10Y        20Y
count  43.000000  39.000000  34.000000  24.000000
mean    0.039436   0.043044   0.046847   0.047412
std     0.064013   0.032239   0.018699   0.012375
min    -0.080814  -0.053359   0.020378   0.026512
25%     0.007254   0.021628   0.033181   0.039997
50%     0.038691   0.045184   0.044814   0.046152
75%     0.071274   0.059059   0.058128   0.055657
max     0.234110   0.130901   0.094228   0.072044
Returns adjusted for inflation and taxes
  

In [346]:
import yfinance as yf
df_com = yf.Ticker('^SPGSCI').history(start='1985-01-01', interval='1mo')
df_com = df_com.iloc[::12, :]
df_com.index = df_com.index.tz_localize(None)
df_com['Return'] = df_com.Close.pct_change()
get_performance([df_com], [1], 'Close', 'Return')

Sharpe ratio
10Y   -0.399474
20Y   -0.272366
dtype: float64


In [347]:
sp500 = yf.Ticker('^GSPC').history(start='1985-01-1', interval='1mo')
sp500 = sp500.iloc[::12, :]
sp500['Return'] = sp500.Close.pct_change()
sp500.index = sp500.index.tz_localize(None)

get_performance([sp500], [1], 'Close', 'Return')
get_performance([sp500, df_bond], [3/5, 2/5], 'Close', 'Return')
get_performance([sp500, df_bond, df_com], [1/3, 1/3, 1/3], 'Close', 'Return')

Sharpe ratio
10Y    0.530893
20Y    1.059661
dtype: float64


Sharpe ratio
10Y    0.462903
20Y    1.234065
dtype: float64
Sharpe ratio
10Y    0.372240
20Y    1.010257
dtype: float64
